In [2]:
import requests
import pandas as pd
import numpy as np
import re 



def converFahrenheit(val):
    return (float(val) - 32) * (5/9)

def processTemp(temp, key):
   value = temp
   if key == "Number":
         temp = temp[key][0]
         numberKey = value.keys() 
         unit = value['Unit'][:2] if "Unit" in numberKey else "°C"
         if "°F" == unit:
            temp = converFahrenheit
   else:
      temp = temp["StringWithMarkup"][0]["String"]
      if "°" in temp:
         temp = temp.partition("°")
      else:
         temp = temp.partition(" ")[0]
         return temp

      val, unit = temp[0], temp[2][0]
      if unit == "F":
         if "to" in val:
            val = val.split("to")
            val = np.random.uniform(float(val[0]), float(val[1]), 1)[0]
         elif "greater than" in val.lower():
            val = val.split("greater than")
         val = converFahrenheit(val)
      temp = val
   return temp 


def processTemp2(temp, key):
   some = temp
   if key == "Number":
         temp = temp[key][0]
         numberKey = some.keys()
         
         unit = some['Unit'][:2] if "Unit" in numberKey else "°C"

         if "°F" == unit:
            temp = converFahrenheit
   else:
      temp = temp["StringWithMarkup"][0]["String"]
      match = re.search(r'(\d+)\s*°?\s*[FfCc]', temp)

      if match:
         temp = match.group(0)
         temp, unit = temp.split(" ")

         if unit == "°F":
            temp = converFahrenheit(temp)
      else:
         temp = processTemp(some, key)
         
   return temp 




In [3]:
path = "C:/Users/Sylvanus/Documents/Project/data/chemicals.xlsx"



data = pd.read_excel(path)
data.dropna(inplace=True)
iterations = data.shape[0]

In [14]:

result = {
   "chemical": [],
   "boilingPoint": [],
   "meltingPoint" : [], 
   "density": []

}

for i in range(iterations):
   chemicalName, cid = data.iloc[i]
   cid = int(cid)
   url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/{cid}/JSON/"
   try:
      r = requests.get(url=url)
      if not r.ok:
         continue
   except TimeoutError as e:
      continue
   
   response = r.json()
       #len(response["Record"]["Section"][3]["Section"][1]["Section"])
   try:
      c = len(response["Record"]["Section"][3]["Section"][1]["Section"])
   except IndexError as e:
      continue
   except KeyError as e:
      continue 

   attributes = []
   for j in range(c):
      attributes.append(response["Record"]["Section"][3]["Section"][1]["Section"][j]["TOCHeading"])
   
   isbp = "Boiling Point" in attributes
   ismp = "Melting Point" in attributes
   isd = "Density" in attributes
   meltingPoint, boilingPoint, density = 0, 0, 0
   # processing boiling point
   if isbp:
      ibp = attributes.index("Boiling Point") 
      boilingPoint = response["Record"]["Section"][3]["Section"][1]["Section"][ibp]["Information"][0]["Value"]
      key = list(boilingPoint.keys())[0]
      boilingPoint = processTemp2(boilingPoint, key)
   else:
      boilingPoint = None

   # processing melting point
   if ismp: 

      imp = attributes.index("Melting Point") 
      meltingPoint = response["Record"]["Section"][3]["Section"][1]["Section"][imp]["Information"][0]["Value"]
      key = list(meltingPoint.keys())[0]
      meltingPoint = processTemp2(meltingPoint, key)
  
   else:
      meltingPoint = None
   
   # Processing density
   if isd:
      id = attributes.index("Density") 
      density = response["Record"]["Section"][3]["Section"][1]["Section"][id]["Information"][0]["Value"]
      key = list(density.keys())[0]

      if key == "Number":
         density = density[key][0]
      else: 
         density = density["StringWithMarkup"][0]["String"]
   
   else:
      density = None
   
   result["chemical"].append(chemicalName)
   result["boilingPoint"].append(boilingPoint)
   result["meltingPoint"].append(meltingPoint)
   result["density"].append(density)




In [15]:

result1 = pd.DataFrame(result)

result1

,chemical,boilingPoint,meltingPoint,density
0,Hydrogen,217.222222,223.333333,"0.071 at -423.4 °F (USCG, 1999) - Less dense t..."
1,Butane,-17.222222,-17.222222,"0.6 at 32 °F (USCG, 1999) - Less dense than wa..."
2,Ozone,-14.444444,156.666667,"1.614 at -319.7 °F (NTP, 1992) - Denser than w..."
3,Ammonium Carbonate,None,58,"1.5 at 68 °F (USCG, 1999) - Denser than water;..."
4,Gallic Acid,SUBLIMES,240.0,"1.7 at 68 °F (USCG, 1999) - Denser than water;..."
...,...,...,...,...
174,Sodium Nitrate,380.0,-16.666667,"2.26 at 68 °F (USCG, 1999) - Denser than water..."
175,Methylene Chloride,-14.444444,-17.222222,"1.322 at 68 °F (USCG, 1999) - Denser than wate..."
176,Aqua Regia,None,None,None
177,Sodium Bicarbonate,None,108.888889,"2.159 (NTP, 1992) - Denser than water; will sink"


In [18]:
def processDensity(density):
    
    if density == None:
       return None
    value = density.partition(" ")[0].split("-")
    

   
    if len(value) > 1:
        value = np.random.uniform(np.float_(value[0]), np.float_(value[1]), 1)[0]
   
    elif (len(value) == 1) and value[0].isalpha():
        match = re.search(r'(\d+\.\d+)', density)
    
        if match:
           value = match.group(1)
    else:
        value = np.float_(value[0])
    
    if "g/l" in density.lower():
        
        density = value * 0.001
    
    else:
        density = value
    
    return density


In [19]:
result1["density"] = result1["density"].apply(processDensity)

In [23]:

result1.dropna(axis=0, how="any", inplace=True)



In [24]:
result1.to_excel("ChemicalInfo.xlsx")